# 修了課題②　MNIST

利用するデータセット：MNIST

合格基準：テストデータに対して正解率95%以上

# データセットのダウンロード


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [13]:
# MNISTデータのロードと訓練データ、検証データの分割
def load_mnist():
    mnist = fetch_openml('MNIST_784', version=1, data_home="MNIST_data/")
    mnist_X, mnist_y = shuffle(mnist.data[:60000].astype('float32'),
                               mnist.target[:60000].astype('int32'), random_state=42)

    # 正規化
    mnist_X = mnist_X / 255.0

    # one-hot表現に変換
    mnist_y = np.eye(10)[mnist_y]

    return train_test_split(mnist_X, mnist_y,
                test_size=0.2,
                random_state=42)

In [26]:
# それぞれのデータの形式を出力
x_train, x_val, y_train, y_val = load_mnist()
x_train = x_train.values
x_val = x_val.values
print('学習データ ', x_train.shape)
print('正解ラベル ', y_train.shape)
print('検証データ ', x_val.shape)
print('正解ラベル ', y_val.shape)

学習データ  (48000, 784)
正解ラベル  (48000, 10)
検証データ  (12000, 784)
正解ラベル  (12000, 10)


# モデルの制作

In [33]:
# ここからダウンロードされたデータに対してモデルを作成し、検証用データに対して予測結果を出力して下さい。
import torch.nn as nn
import torch

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNormを追加
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNormを追加
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)  # Dropoutを調整
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
        # Optimizerと損失関数を定義
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  # MNISTの画像を2D形式に変換
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))
        return x
    
    def optimize(self, x, y):
        self.optimizer.zero_grad()
        output = self.forward(x)
        loss = self.criterion(output, y)
        loss.backward()
        self.optimizer.step()
        return loss

# モデルのインスタンスを生成
model = CNN() 

In [34]:
# モデルで学習を行う
# Epoch数を指定
num_epochs = 10
batch_size = 128

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(x_train), batch_size):
        x_batch = torch.FloatTensor(x_train[i:i+batch_size])
        y_batch = torch.argmax(torch.FloatTensor(y_train[i:i+batch_size]), dim=1)
        loss = model.optimize(x_batch, y_batch)
    
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.FloatTensor(x_val))
        val_loss = model.criterion(val_outputs, torch.argmax(torch.FloatTensor(y_val), dim=1))
        accuracy = (torch.argmax(val_outputs, dim=1) == torch.argmax(torch.FloatTensor(y_val), dim=1)).sum().item() / len(y_val)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}, Val Loss: {val_loss.item():.4f}, Accuracy: {accuracy:.4f}')

Epoch [1/10], Loss: 1.5045, Val Loss: 1.5019, Accuracy: 0.9608
Epoch [2/10], Loss: 1.4939, Val Loss: 1.4911, Accuracy: 0.9718
Epoch [3/10], Loss: 1.4952, Val Loss: 1.4840, Accuracy: 0.9785
Epoch [4/10], Loss: 1.4904, Val Loss: 1.4806, Accuracy: 0.9810
Epoch [5/10], Loss: 1.4946, Val Loss: 1.4780, Accuracy: 0.9831
Epoch [6/10], Loss: 1.4968, Val Loss: 1.4787, Accuracy: 0.9828
Epoch [7/10], Loss: 1.4861, Val Loss: 1.4784, Accuracy: 0.9823
Epoch [8/10], Loss: 1.4800, Val Loss: 1.4758, Accuracy: 0.9858
Epoch [9/10], Loss: 1.4782, Val Loss: 1.4751, Accuracy: 0.9863
Epoch [10/10], Loss: 1.4771, Val Loss: 1.4740, Accuracy: 0.9873


# 提出形式

## テスト用データセットのダウンロード

In [36]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# テスト用のデータ（正解ラベルなし）のダウンロード
!wget 'https://drive.google.com/uc?export=download&id=18h1BsXvC6q_yZsO_TcONyotw5wXEU9b8' -O mnist_x_test.csv

--2025-01-26 21:58:27--  https://drive.google.com/uc?export=download&id=18h1BsXvC6q_yZsO_TcONyotw5wXEU9b8
正在解析主机 drive.google.com (drive.google.com)... 142.251.222.14
正在连接 drive.google.com (drive.google.com)|142.251.222.14|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 303 See Other
位置：https://drive.usercontent.google.com/download?id=18h1BsXvC6q_yZsO_TcONyotw5wXEU9b8&export=download [跟随至新的 URL]
--2025-01-26 21:58:28--  https://drive.usercontent.google.com/download?id=18h1BsXvC6q_yZsO_TcONyotw5wXEU9b8&export=download
正在解析主机 drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.207.33
正在连接 drive.usercontent.google.com (drive.usercontent.google.com)|142.250.207.33|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：34001360 (32M) [application/octet-stream]
正在保存至: “mnist_x_test.csv”

mnist_x_test.csv    100%[===================>]  32.43M  6.80MB/s  用时 4.8s      

2025-01-26 21:58:36 (6.75 MB/s) - 已保存 “mnist_x_test.csv” [34001360/34001360])



In [37]:
# テスト用データの確認
x_test = pd.read_csv('mnist_x_test.csv', index_col=0)
x_test.values.shape

(10000, 784)

In [42]:
# 提出形式見本
# 提出データの形、タイトル
y_pred = model.forward(torch.FloatTensor(x_test.values)).detach().numpy()
y_pred = np.argmax(y_pred, axis=1)
y_pred = pd.DataFrame(y_pred, columns=['Number'])
# csv形式で提出して下さい。
y_pred.to_csv('y_pred.csv')
y_pred

,Number
0,7
1,2
2,1
3,0
4,4
...,...
9995,2
9996,3
9997,4
9998,5
